In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

import lightgbm as lgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, max_error

import pickle
import requests

In [2]:
url = 'https://api.open-meteo.com/v1/forecast'
body = {
    "latitude": 16.4328,
    "longitude": 103.5066,
    "daily": 'weathercode,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,uv_index_max,uv_index_clear_sky_max,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration',
    "timezone": 'auto',
    "forecast_days": 14
}
r = requests.get(url, params=body, verify=False)
payload = r.json()

In [3]:
df = pd.DataFrame(payload['daily'])
df = df[['time','temperature_2m_max', 'apparent_temperature_max', 'precipitation_sum', 'rain_sum', 'precipitation_hours', 'shortwave_radiation_sum', 'et0_fao_evapotranspiration']]
df 

,time,temperature_2m_max,apparent_temperature_max,precipitation_sum,rain_sum,precipitation_hours,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2023-10-05,32.3,38.5,11.8,0.0,23.0,14.86,3.19
1,2023-10-06,29.7,35.7,7.5,0.0,19.0,11.57,2.46
2,2023-10-07,30.0,35.7,19.5,0.0,23.0,12.41,2.62
3,2023-10-08,29.5,35.7,11.8,0.0,17.0,11.33,2.37
4,2023-10-09,31.7,34.9,3.0,0.0,6.0,14.68,3.34
5,2023-10-10,32.1,35.7,0.0,0.0,0.0,20.70,4.51
6,2023-10-11,31.7,35.1,0.0,0.0,0.0,20.62,4.59
7,2023-10-12,29.7,33.1,1.2,0.6,6.0,14.40,3.21
8,2023-10-13,29.4,34.4,3.6,1.8,9.0,18.19,3.75
9,2023-10-14,25.2,28.7,0.0,0.0,0.0,3.84,1.11


In [4]:
with open('../datasets/lgbm_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [1]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)